# Policy Learning Only Workflow

In this workflow, CFRL takes in an offline trajectory and then preprocesses the offline trajectory using `SequentialPreprocessor`. After that, the preprocessed trajectory is passed into `FQI` to train a counterfactually fair policy, which is the final output of the workflow. This workflow is appropriate if the user wants to train a policy using CFRL. The trained policy can be further evaluated on its value and counterfactual fairness, which is discussed in detail in the "Assessing Policies Using Real Data" notebook.

We begin by importing the liberaries needed for this demonstration.

In [1]:
# Need this temporarily to import CFRL before it is officially published to PyPI
import sys
sys.path.append("E:/learning/university/MiSIL/CFRL Python Package/CFRL")

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from cfrl.reader import read_trajectory_from_dataframe, convert_trajectory_to_dataframe
from cfrl.preprocessor import SequentialPreprocessor
from cfrl.agents import FQI
np.random.seed(10) # ensure reproducibility
torch.manual_seed(10) # ensure reproducibility

## Data Loading

In this demonstration, we use an offline trajectory generated from a `SyntheticEnvironment` using some pre-specified transition rules. Although it is actually synthesized, we treat it as if it is from some unknown environment for pedagogical convenience in this demonstration.

The trajectory contains 500 individuals (i.e. $N=500$) and 10 transitions (i.e. $T=10$). The actions are binary (0 or 1) and were sampled using a random policy that selects 0 or 1 randomly with equal probability. It is stored in a tabular format in a `.csv` file. The sensitive attribute variable is bivariate, stored in columns `z1` and `z2`. The legit values of the sensitive attribute are $[0, 0]$, $[1, 0]$, $[0, 1]$, and $[1, 1]$. The state variable is tri-variate, stored in columns `state1`, `state2`, and `state3`. The actions are stored in the column `action` and rewards in the column `reward`. The tabular data also includes an extra irrelevant column `timestamp`. 

We can load and view the tabular data.

In [3]:
trajectory = pd.read_csv('../data/sample_data_large_multi_2.csv')
trajectory

,Unnamed: 0,ID,timestamp,z1,z2,action,reward,state1,state2,state3
0,0,1.0,1.0,1.0,1.0,NaN,NaN,4.124345,1.888244,1.971828
1,1,1.0,2.0,1.0,1.0,1.0,3.789688,4.682275,3.510520,3.011858
2,2,1.0,3.0,1.0,1.0,0.0,4.336946,2.471132,1.272698,1.540864
3,3,1.0,4.0,1.0,1.0,0.0,1.295495,1.456000,1.386428,1.160023
4,4,1.0,5.0,1.0,1.0,1.0,1.912048,2.154422,1.431473,1.264020
...,...,...,...,...,...,...,...,...,...,...
5495,5495,500.0,7.0,1.0,1.0,0.0,-0.289759,1.912800,-1.068690,0.615442
5496,5496,500.0,8.0,1.0,1.0,0.0,1.389520,0.695095,-0.697456,1.780840
5497,5497,500.0,9.0,1.0,1.0,0.0,0.389651,0.817572,-0.199511,1.429284
5498,5498,500.0,10.0,1.0,1.0,0.0,0.029739,-0.598246,-0.371662,0.518159


We now read the trajectory from the tabular format into Trajectory Arrays.

In [4]:
zs, states, actions, rewards, ids = read_trajectory_from_dataframe(
                                                data=trajectory, 
                                                z_labels=['z1', 'z2'], 
                                                state_labels=['state1', 'state2', 'state3'], 
                                                action_label='action', 
                                                reward_label='reward', 
                                                id_label='ID', 
                                                T=10
                                                )

## Preprocessor Training

Before preprocessing the trajectory, we need to first train a preprocessor. To mitigate overfitting, we use a random subset of 250 individuals in the trajectory to train the preprocessor. The remaining 250 individuals will be actually preprocessed. We now form these two sets.

In [5]:
(
    zs_train, zs_prepro, 
    states_train, states_prepro, 
    actions_train, actions_prepro, 
    rewards_train, rewards_prepro, 
    ids_train, ids_prepro
) = train_test_split(zs, states, actions, rewards, ids, test_size=0.5)

We now use the training set to train a `SequentialPreprocessor`.

In [6]:
sp = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                            num_actions=2, 
                            cross_folds=1, 
                            mode='single', 
                            reg_model='nn')
sp.train_preprocessor(zs=zs_train, xs=states_train, actions=actions_train, rewards=rewards_train)

100%|██████████| 1000/1000 [00:34<00:00, 28.78it/s]


(array([[[ 1.47487332e-01,  1.50254830e+00, -1.66469333e+00, ...,
           2.05579699e+00,  3.49205066e+00,  5.78904901e-01],
         [ 1.54685410e-01,  2.75040511e-01, -1.49451782e+00, ...,
           2.73385984e+00,  2.65295518e+00,  6.04489106e-01],
         [-1.32980964e+00, -1.11267430e+00, -1.99076036e+00, ...,
           2.85810735e+00,  6.95762332e-01, -4.31892000e-02],
         ...,
         [-5.66250520e-02, -8.48483132e-01, -1.17231567e+00, ...,
           4.76362781e-01,  3.89372482e-01, -8.59598406e-01],
         [-1.97910154e+00, -8.23400731e-01, -1.78218275e+00, ...,
          -1.01817249e-01,  4.91883402e-01, -3.73013399e-01],
         [-5.98881224e-01, -1.19249451e+00, -2.65340119e+00, ...,
           1.33478369e+00, -8.91307651e-01, -2.23343658e+00]],
 
        [[ 8.49102267e-01,  1.17672025e+00, -5.48338858e-01, ...,
           2.75741192e+00,  3.16622261e+00,  1.69525937e+00],
         [-1.93749297e+00,  2.51739130e-01, -2.68465126e+00, ...,
          -8.19620554

## Policy Learning

We now train a counterfactually fair policy using the preprocessor `sp` and the set of trajectory data that is not used to train `sp`. We begin by initializing an `FQI` agent with `sp` as its internal preprocessor.

In [7]:
agent = FQI(num_actions=2, model_type='nn', preprocessor=sp)

We now perform training. Since we set `preprocess=True` in `train()`, `agent` will use its internal preprocessor (i.e. `sp`) to automatically preprocess the input training trajectory before using the trajectory for policy learning. Therefore, we can directly pass in the unpreprocessed `states_prepro` and `rewards_prepro`. 

In [8]:
agent.train(zs=zs_prepro, 
            xs=states_prepro, 
            actions=actions_prepro, 
            rewards=rewards_prepro, 
            max_iter=100, 
            preprocess=True)

100%|██████████| 100/100 [01:10<00:00,  1.42it/s]


## Decision-making Using the Trained Policy

Now we have trained a counterfactually fair policy. We can use its `act()` method to make decisions. At actual deployment, the single-step trajectory passed into `act()` is often generated from the real enviornment at the time. However, in this part, we reuse the training trajectory data for demonstration purposes.

We first take actions for the initial time step (i.e. $a_0$). Again, we only pass in unpreprocessed states and rewards because they will be automatically preprocessed inside `act()` before being used for decision-making. We set `xtm1` and `atm1` to `None`. This tells `agent` that we are taking the initial action and that the buffer of the internal preprocessor should be reset.

In [9]:
a0 = agent.act(z=zs, 
               xt=states[:, 0], 
               xtm1=None, 
               atm1=None, 
               preprocess=True)
a0

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

We now take the second action. We still pass in unpreprocessed states and rewards, and we specify non-`None` `xtm1` and `atm1` to tell `agent` not to reset the buffer of its internal preprocessor.

In [10]:
a1 = agent.act(z=zs, 
               xt=states[:, 1], 
               xtm1=states[:, 0], 
               atm1=a0, 
               preprocess=True)
a1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## Alternative: Using All Individuals for Policy Learning

Sometimes, the number of individuals in the trajectory is small. In this case, if we only use a subset of individuals for policy learning, the amount of training trajectory data might be too small to be useful for policy learning. Fortunately, when we set `cross_folds` to a relatively large number, we can directly preprocess all individuals using the `train_preprocessor()` function, and the preprocessed trajectory can be used for policy learning.

When `cross_folds=K` where `K` is greater than 1, `train_preprocessor()` will internally divide the training data into `K` folds. For each $i=1,\dots,k$, it trains a transition dynamics model based on all the folds other than the $i$-th one, and this model is then used to preprocess data in the $i$-th fold. This results in `K` folds of preprocessed data, each of which is processed using a model that is trained on the other folds. These `K` folds of preprocessed data are then combined and returned by `train_preprocessor()`. This method allows us to preprocess all individuals in the trajectory while reducing overfitting.

To use this functionality, we first initialize a `SequentialPreprocessor` with `cross_folds` greater than 1. We use `cross_folds=5` here.

In [11]:
sp_cf5 = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                                num_actions=2, 
                                cross_folds=5, 
                                mode='single', 
                                reg_model='nn')

We now simultaneously train the preprocessor and preprocess all individuals in the trajectory using the precedure described above.

In [12]:
states_tilde_cf5, rewards_tilde_cf5 = sp_cf5.train_preprocessor(zs=zs, 
                                                                xs=states, 
                                                                actions=actions, 
                                                                rewards=rewards)

100%|██████████| 1000/1000 [00:41<00:00, 24.16it/s]


We now train a policy using `FQI` and the preprocessed data. Note that we set `preprocess=False` during training because the input training data `state_tilde` and `rewards_tilde` are already preprocessed.

In [13]:
agent_cf5 = FQI(num_actions=2, model_type='nn', preprocessor=sp)
agent_cf5.train(zs=zs, 
                xs=states_tilde_cf5, 
                actions=actions, 
                rewards=rewards_tilde_cf5, 
                max_iter=100, 
                preprocess=False)

100%|██████████| 100/100 [01:09<00:00,  1.44it/s]


We now take the initial action using `agent_cf5`.

In [14]:
a0 = agent_cf5.act(z=zs, 
               xt=states[:, 0], 
               xtm1=None, 
               atm1=None, 
               preprocess=True)
a0

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

We now take the second action using `agent_cf5`.

In [15]:
a1 = agent_cf5.act(z=zs, 
               xt=states[:, 1], 
               xtm1=states[:, 0], 
               atm1=a0, 
               preprocess=True)
a1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,